In [1]:
# Import libraries

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import glob
import os
import datetime
from datetime import timedelta

# Import plotly express for EF plot
import plotly.express as px
import plotly.graph_objects as go
px.defaults.width, px.defaults.height = 1000,600
# Set precision
pd.set_option('display.precision', 4)

In [2]:
df2 = pd.read_csv('options_final_total_1.csv', index_col=0)

In [3]:
# #Read the CSV file
# dates = pd.read_excel('E:\marker4dates.xlsx')

# #Extract the 'Date' column (replace 'Date' with the actual column name in your CSV)
# dates_to_plot = dates['date'].astype(str).tolist()

In [4]:
# dates_to_plot = pd.to_datetime(dates_to_plot).date
# # Filter the DataFrame for the specified dates
# df2 = df2[df2['date'].isin(dates_to_plot)]

In [5]:
# df2 = df2[df2['date'] >= pd.to_datetime('2021-01-01').date()]
# del df2['date']

In [6]:
# Convert 'quote_datetime' to datetime
df2['quote_datetime'] = pd.to_datetime(df2['quote_datetime'])
df2['date'] = df2['quote_datetime'].dt.date
df2['time'] = df2['quote_datetime'].dt.time

In [7]:
df2 = df2[df2['quote_datetime'].dt.date != pd.to_datetime('2020-03-09').date()]

In [8]:
df2 = df2[df2['quote_datetime'].dt.date != pd.to_datetime('2020-09-11').date()]

In [10]:
# Filter rows where the time falls between 9:35 AM and 4:10 PM
start_time = pd.to_datetime('09:32:00').time()
end_time = pd.to_datetime('10:32:00').time()

# Apply the filter based on the time component of 'quote_datetime'
df3 = df2[(df2['quote_datetime'].dt.time >= start_time) & (df2['quote_datetime'].dt.time <= end_time)]

In [11]:
# Calculate the number of unique days
num_unique_days = df3['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [12]:
df3.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932'],
      dtype='object')

In [13]:
def exit_strategy(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'Close' value for each date (first_close)
        first_close = group['Close'].iloc[0]
        first_straddle = group['put_price'].iloc[0]
        
        # Step 2: Calculate the condition and filter the rows accordingly
        condition = group['put_price'] < 1.7 * first_straddle
        
        # Step 3: Find the first row where the condition fails and filter accordingly
        if condition.any():
            first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
            # Keep rows before the first failure (inclusive of the row where it fails)
            if first_fail_idx:
                filtered_group = group.loc[:first_fail_idx]
            else:
                filtered_group = group  # Keep the entire group if the condition never fails
            
            filtered_df_list.append(filtered_group)
    
    return pd.concat(filtered_df_list, ignore_index=True)

In [14]:
# Step 5: Apply the exit strategy
df_filtered = exit_strategy(df3)

In [15]:
# Calculate the number of unique days
num_unique_days = df_filtered['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [16]:
df_filtered['time'] = df_filtered['quote_datetime'].dt.time

In [17]:
def exit_strategy1(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_straddle = group['call_price'].iloc[0]
        first_put = group['put_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1.25 * first_straddle
        condition = 1.5 * first_put
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        condition_triggered = False
        group['putprice1stop'] = np.nan
        group['callprice1stop'] = np.nan
        group['time1stop'] = np.nan
        
        for i in range(len(group)):
            if not stop_loss_triggered and group['call_price'].iloc[i] >= stop_loss_level:
                stop_loss_triggered = True
                b = i

            if not condition_triggered and group['put_price'].iloc[i] >= condition:
                if not stop_loss_triggered:
                    condition_triggered = True
                    a = i
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                #group.loc[group.index[i], 'call_price'] = stop_loss_level
                #group['stoploss'] = True
                group['putprice1stop'] = group.loc[group.index[b], 'put_price']
                group['callprice1stop'] = group.loc[group.index[b], 'call_price']
                group['time1stop'] = group.loc[group.index[b], 'time']
            elif condition_triggered:
                group.loc[group.index[i], 'call_price'] = group.loc[group.index[a], 'call_price']
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [18]:
# Step 5: Apply the exit strategy
df_filtered1 = exit_strategy1(df_filtered)

In [19]:
# Calculate the number of unique days
num_unique_days = df_filtered1['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [24]:
df_filtered1.head()

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,strike932,date,time,call932,put932,straddle932,spot932,putprice1stop,callprice1stop,time1stop
0,2018-01-02 09:32:00,2685.0,3.30,4.3,3.00,3.9,2685.60,2685.60,2684.30,2684.30,...,2685.0,2018-01-02,09:32:00,3.15,4.1,7.25,2684.3,2.625,4.35,09:41:00
1,2018-01-02 09:33:00,2685.0,3.90,3.9,3.40,3.4,2684.32,2685.74,2684.23,2685.30,...,2685.0,2018-01-02,09:33:00,3.15,4.1,7.25,2684.3,2.625,4.35,09:41:00
2,2018-01-02 09:34:00,2685.0,3.10,4.5,2.70,4.2,2685.18,2685.18,2683.73,2683.73,...,2685.0,2018-01-02,09:34:00,3.15,4.1,7.25,2684.3,2.625,4.35,09:41:00
3,2018-01-02 09:35:00,2685.0,2.75,5.0,2.45,4.5,2683.80,2684.18,2683.05,2683.07,...,2685.0,2018-01-02,09:35:00,3.15,4.1,7.25,2684.3,2.625,4.35,09:41:00
4,2018-01-02 09:36:00,2685.0,2.65,5.0,2.55,4.7,2683.06,2683.23,2682.78,2682.94,...,2685.0,2018-01-02,09:36:00,3.15,4.1,7.25,2684.3,2.625,4.35,09:41:00


In [25]:
# import pandas as pd

# def exit_strategy10(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Step 1: Get the first 'Close' value for each date (first_close)
#         if group['time1stop'].isna().all():
#             filtered_group = group
#         else:
#             first_close = group['Close'].iloc[0]
#             first_straddle = group['put_price'].iloc[0]
            
#             # Step 2: Filter rows where time > time1stop
#             valid_rows = group[group['time'] > group['time1stop']]
            
#             # Step 3: Calculate the condition and filter the rows accordingly
#             condition = valid_rows['put_price'] < 1.3 * first_straddle
            
#             # Step 4: Find the first row where the condition fails and filter accordingly
#             if condition.any():
#                 first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
                
#                 # Keep rows before the first failure (inclusive of the row where it fails)
#                 if first_fail_idx is not None:
#                     filtered_group = group.loc[:first_fail_idx]
#                 else:
#                     filtered_group = group  # Keep the entire group if the condition never fails
#             else:
#                 filtered_group = group  # If no valid rows, keep the entire group
            
#         filtered_df_list.append(filtered_group)
    
#     return pd.concat(filtered_df_list, ignore_index=True)

In [26]:
# # Step 5: Apply the exit strategy
# df_filtered1 = exit_strategy10(df_filtered1)

In [27]:
# Calculate the number of unique days
num_unique_days = df_filtered1['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [28]:
# def exit_strategy2(df):
#     filtered_df_list = []
    
#     # Group by each date
#     for date, group in df.groupby('date'):
#         # Step 1: Get the first 'Close' value for each date (first_close)
#         #first_sma = group['intraday_sma_21'].iloc[0]
#         stoploss = group['stoploss'].iloc[0]
        
#         # Step 2: Calculate the condition and filter the rows accordingly
#         condition = (group['Close'] > group['intraday_sma_21']) | (group['quote_datetime'].dt.time < pd.to_datetime('09:51').time())
        
#         # Step 3: Find the first row where the condition fails and filter accordingly
#         if condition.any():
#             first_fail_idx = condition[~condition].index[0] if (~condition).any() else None
            
#             # Keep rows before the first failure (inclusive of the row where it fails)
#             if first_fail_idx and stoploss:
#                 filtered_group = group.loc[:first_fail_idx+1]
#             else:
#                 filtered_group = group  # Keep the entire group if the condition never fails
            
#             filtered_df_list.append(filtered_group)
    
#     return pd.concat(filtered_df_list, ignore_index=True)

In [30]:
# # Step 5: Apply the exit strategy
# df_filtered2 = exit_strategy2(df_filtered)

In [31]:
df_filtered2 = df_filtered1.copy()

In [32]:
# Calculate the number of unique days
num_unique_days = df_filtered2['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [34]:
# df_filtered2['call_price'] /= 2

In [35]:
df_filtered2['straddle_exit'] = df_filtered2['call_price'] + df_filtered2['put_price']

In [37]:
df_filtered2['spot932'] = df_filtered2['Close'].where(df_filtered2['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
df_filtered2['spot932'] = df_filtered2['spot932'].ffill() 

# df_filtered2['spot932'] = df_filtered2.groupby(df_filtered2['quote_datetime'].dt.date)['Close'].transform('first').ffill()

In [38]:
# df_filtered2['put932'] = df_filtered2['put_price'].where(df_filtered2['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
# df_filtered2['put932'] = df_filtered2['put932'].ffill() 

In [39]:
# df_filtered2['call932'] = df_filtered2['call_price'].where(df_filtered2['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
# df_filtered2['call932'] = df_filtered2['call932'].ffill()    

In [40]:
df_filtered2['straddle932'] = df_filtered2['straddle_exit'].where(df_filtered2['quote_datetime'].dt.time == pd.to_datetime('09:32:00').time())
df_filtered2['straddle932'] = df_filtered2['straddle932'].ffill()    

In [42]:
df_filtered2['call40%'] = (df_filtered2['call_price'] - df_filtered2['call932'])/df_filtered2['call932'] > 0.4

In [46]:
# Filter rows where 'time1stop' is not NaN
df_filtered_not = df_filtered2[df_filtered2['time1stop'].notna()]

In [47]:
# Filter rows where 'time1stop' is NaN or equals 10:32
df_filtered_is = df_filtered2[
    df_filtered2['time1stop'].isna() | (df_filtered2['time1stop'] == pd.to_datetime('10:32').time())
]

In [48]:
# Calculate the number of unique days
num_unique_days = df_filtered_is['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

606


In [49]:
df_filtered_not = df_filtered_not[df_filtered_not['time'] > df_filtered_not['time1stop']]

In [54]:
# Calculate the number of unique days
num_unique_days = df_filtered_not['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

713


In [55]:
# Ensure the DataFrame is sorted by 'date' and 'time'
df_filtered_not = df_filtered_not.sort_values(by=['date', 'time'])
df_filtered_not['call_below_1st'] = False

# Define a function to filter each group based on the first occurrence of a condition
def filter_first_true(group):
    # Check if the 'call40%' column has any True values
    if group['call40%'].any():
        return group  # If True is found, return the group without filtering
    
    # Otherwise, find the index of the first occurrence where 'call_price' <= 'call932'
    first_yes_idx = group[group['call_price'] <= group['call932']].index.min()
    
    # If such an index exists, filter the group to keep rows from this index onward
    if not pd.isna(first_yes_idx):
        group = group.loc[first_yes_idx:]
        group['call_below_1st'] = True
    
    return group

# Apply the filtering function to each group based on 'date'
df_filtered_not1 = df_filtered_not.groupby('date').apply(filter_first_true).reset_index(drop=True)

In [56]:
# df_filtered_not2

In [57]:
df_filtered_not1

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,time,call932,put932,straddle932,spot932,putprice1stop,callprice1stop,time1stop,call40%,call_below_1st
0,2018-01-02 09:42:00,2685.0,4.70,2.60,3.90,2.40,2686.56,2686.99,2686.41,2686.80,...,09:42:00,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False
1,2018-01-02 09:43:00,2685.0,4.70,2.55,4.20,2.35,2686.84,2687.13,2686.55,2686.95,...,09:43:00,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,True,False
2,2018-01-02 09:44:00,2685.0,4.30,2.80,3.60,2.55,2686.96,2686.99,2686.24,2686.38,...,09:44:00,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False
3,2018-01-02 09:45:00,2685.0,4.20,2.95,3.70,2.70,2686.39,2686.72,2685.92,2686.19,...,09:45:00,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False
4,2018-01-02 09:46:00,2685.0,3.60,3.40,3.10,3.30,2686.09,2686.35,2685.17,2685.17,...,09:46:00,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28736,2024-11-27 10:28:00,6015.0,3.90,14.00,3.90,14.00,6006.45,6006.45,6003.96,6004.26,...,10:28:00,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True
28737,2024-11-27 10:29:00,6015.0,4.50,12.60,4.50,12.60,6004.23,6006.02,6003.53,6005.92,...,10:29:00,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True
28738,2024-11-27 10:30:00,6015.0,3.95,13.20,3.95,13.20,6005.94,6006.18,6004.58,6004.58,...,10:30:00,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True
28739,2024-11-27 10:31:00,6015.0,4.25,12.90,4.25,12.90,6004.77,6005.59,6003.80,6005.37,...,10:31:00,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True


In [58]:
# Calculate the number of unique days
num_unique_days = df_filtered_not1['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

713


In [59]:
# # Calculate the number of unique days
# num_unique_days = df_filtered_not2['quote_datetime'].dt.date.nunique()

# # Display the result
# print(num_unique_days)

In [60]:
# Define a function to extract the first 'call_price' and forward fill it
def add_first_call_price(group):
    # Extract the first row's 'call_price' for the group
    first_call_price = group['call_price'].iloc[0]
    
    # Create a new column with the first 'call_price' value, forward filled
    group['call_price2sell'] = first_call_price
    
    # Forward fill the new column within the group
    group['call_price2sell'] = group['call_price2sell'].ffill()
    
    return group

# Apply the function to each group based on 'date'
df_filtered_not1 = df_filtered_not1.groupby('date').apply(add_first_call_price).reset_index(drop=True)

In [61]:
df_filtered_not1

,quote_datetime,strike,ask_C,ask_P,bid_C,bid_P,Open,High,Low,Close,...,call932,put932,straddle932,spot932,putprice1stop,callprice1stop,time1stop,call40%,call_below_1st,call_price2sell
0,2018-01-02 09:42:00,2685.0,4.70,2.60,3.90,2.40,2686.56,2686.99,2686.41,2686.80,...,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False,4.3
1,2018-01-02 09:43:00,2685.0,4.70,2.55,4.20,2.35,2686.84,2687.13,2686.55,2686.95,...,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,True,False,4.3
2,2018-01-02 09:44:00,2685.0,4.30,2.80,3.60,2.55,2686.96,2686.99,2686.24,2686.38,...,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False,4.3
3,2018-01-02 09:45:00,2685.0,4.20,2.95,3.70,2.70,2686.39,2686.72,2685.92,2686.19,...,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False,4.3
4,2018-01-02 09:46:00,2685.0,3.60,3.40,3.10,3.30,2686.09,2686.35,2685.17,2685.17,...,3.15,4.10,7.25,2684.30,2.625,4.35,09:41:00,False,False,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28736,2024-11-27 10:28:00,6015.0,3.90,14.00,3.90,14.00,6006.45,6006.45,6003.96,6004.26,...,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True,8.8
28737,2024-11-27 10:29:00,6015.0,4.50,12.60,4.50,12.60,6004.23,6006.02,6003.53,6005.92,...,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True,8.8
28738,2024-11-27 10:30:00,6015.0,3.95,13.20,3.95,13.20,6005.94,6006.18,6004.58,6004.58,...,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True,8.8
28739,2024-11-27 10:31:00,6015.0,4.25,12.90,4.25,12.90,6004.77,6005.59,6003.80,6005.37,...,9.20,11.21,20.41,6011.55,7.600,12.60,09:34:00,False,True,8.8


In [62]:
df_filtered_not12 = df_filtered_not1[df_filtered_not1['call_below_1st'] == True]

In [63]:
df_filtered_not11 = df_filtered_not1[df_filtered_not1['call_below_1st'] == False]

In [64]:
def exit_strategy2(df):
    filtered_df_list = []
    
    # Group by each date
    for date, group in df.groupby('date'):
        # Step 1: Get the first 'call_pnl' value for the date (first_straddle)
        first_straddle = group['call_price'].iloc[0]
        first_put = group['put_price'].iloc[0]
        
        # Step 2: Define the stop-loss level
        stop_loss_level = 1.25 * first_straddle
        condition = 1.5 * first_put
        
        # Step 3: Apply the stop-loss logic
        stop_loss_triggered = False  # Track if the stop-loss condition has been hit
        condition_triggered = False
        group['callprice2stop'] = np.nan
        
        for i in range(len(group)):
            if not stop_loss_triggered and group['call_price'].iloc[i] >= stop_loss_level:
                stop_loss_triggered = True
                #b = i

            if not condition_triggered and group['put_price'].iloc[i] >= condition:
                if not stop_loss_triggered:
                    condition_triggered = True
                    a = i
            
            # Forward-fill the stop-loss level if triggered
            if stop_loss_triggered:
                group.loc[group.index[i], 'call_price'] = stop_loss_level
                group['callprice2stop'] = stop_loss_level
            elif condition_triggered:
                group.loc[group.index[i], 'call_price'] = group.loc[group.index[a], 'call_price']
        
        # Step 4: Add the adjusted group to the filtered list
        filtered_df_list.append(group)
    
    # Combine all the filtered groups
    return pd.concat(filtered_df_list, ignore_index=True)

In [65]:
# Step 5: Apply the exit strategy
df_filtered_not1 = exit_strategy2(df_filtered_not12)

In [66]:
# df_filtered_not1.to_csv('ch.csv')

In [67]:
df_filtered_not1.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'putprice1stop', 'callprice1stop', 'time1stop', 'call40%',
       'call_below_1st', 'call_price2sell', 'callprice2stop'],
      dtype='object')

In [68]:
del df_filtered_not1['call40%']
del df_filtered_not1['call_below_1st']

In [69]:
df_filtered_not11.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'putprice1stop', 'callprice1stop', 'time1stop', 'call40%',
       'call_below_1st', 'call_price2sell'],
      dtype='object')

In [70]:
del df_filtered_not11['call40%']
del df_filtered_not11['call_below_1st']
del df_filtered_not11['call_price2sell']

df_filtered_not11['call_price2sell'] = np.nan
df_filtered_not11['callprice2stop'] = np.nan

In [71]:
df_filtered_is['call_price2sell'] = np.nan
df_filtered_is['callprice2stop'] = np.nan

In [72]:
final_df = pd.concat([df_filtered_is, df_filtered_not1, df_filtered_not11], ignore_index=True)

In [73]:
final_df['straddle_exit'] = final_df['call_price'] + final_df['put_price']

In [74]:
final_df = final_df.sort_values(by='quote_datetime').reset_index(drop=True)

In [75]:
final_df['put_pnl'] = final_df['put932'] - final_df['put_price']
final_df['call_pnl'] = final_df['call932'] - final_df['call_price']
final_df['straddle_pnl'] = final_df['straddle932'] - final_df['straddle_exit']

In [76]:
df_filtered2['time'] = df_filtered2['quote_datetime'].dt.time

In [77]:
final_df['straddle_pnl_bps'] =  (final_df['straddle_pnl']/ final_df['spot932'])*10000
final_df['put_pnl_bps'] =  (final_df['put_pnl']/ final_df['spot932'])*10000
final_df['call_pnl_bps'] =  (final_df['call_pnl']/ final_df['spot932'])*10000

In [78]:
final_df.columns

Index(['quote_datetime', 'strike', 'ask_C', 'ask_P', 'bid_C', 'bid_P', 'Open',
       'High', 'Low', 'Close', 'day_close', 'prev_close', 'gap',
       'prev_day_neg', 'range', 'realized_volatility_post30min',
       'realized_volatility_full_day', 'sma_50', 'sma_21', 'below_sma_50',
       'intraday_sma_21', 'gap931', 'date1', 'high931', 'low931', 'close931',
       'open931', 'straddle_exit', 'call_price', 'put_price', 'strike932',
       'date', 'time', 'call932', 'put932', 'straddle932', 'spot932',
       'putprice1stop', 'callprice1stop', 'time1stop', 'call40%',
       'call_price2sell', 'callprice2stop', 'put_pnl', 'call_pnl',
       'straddle_pnl', 'straddle_pnl_bps', 'put_pnl_bps', 'call_pnl_bps'],
      dtype='object')

In [79]:
df1 = final_df[['quote_datetime', 'date', 'time', 'strike932', 'low931', 'high931', 'open931', 'close931', 'Close', 'day_close', 'spot932', 'gap931', 'realized_volatility_post30min', 'prev_close', 'gap', 'prev_day_neg', 'range', 'realized_volatility_full_day', 'sma_50', 'sma_21', 'intraday_sma_21', 'below_sma_50', 'straddle_exit', 'straddle932', 'straddle_pnl', 'straddle_pnl_bps', 'put_price', 'put932', 'put_pnl', 'put_pnl_bps', 'call_price', 'call932', 'call_pnl', 'call_pnl_bps', 'callprice1stop', 'putprice1stop', 'time1stop', 'call_price2sell', 'callprice2stop']]

In [80]:
# df1['straddleinpercent'] = (df1['straddle932']/df1['spot932'])*100

In [81]:
# df1['rangebystraddle'] = (df1['range']/df1['straddle932'])
# df1['gapbystraddle'] = (df1['gap931']/df1['straddle932'])

In [82]:
# Calculate the number of unique days
num_unique_days = df1['quote_datetime'].dt.date.nunique()

# Display the result
print(num_unique_days)

1319


In [83]:
df1['year'] = df1['quote_datetime'].dt.year

In [84]:
df2 = df1.groupby('date').tail(1).reset_index(drop=True)

In [85]:
df2.to_excel('marker3_recheck3.xlsx')